In [1]:
import torch
from torchdrug import datasets

# dataset = datasets.ZINC250k("~/molecule-datasets/", kekulize=True,
#                             atom_feature="symbol")

In [ ]:
from torch_geometric.utils import degree
from pna import PNA

In [ ]:
deg = torch.zeros(10, dtype=torch.long)
for data in dataset:
    graph = data['graph']
    d = degree(graph.edge_list[:, 1], num_nodes=graph.num_node, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

PNA(input_dim=dataset.node_feature_dim,
                  hidden_dim=256, num_layer=3,
                  edge_input_dim=dataset.edge_feature_dim,
                  num_relation=dataset.num_bond_type,
                  aggregators=['mean'],
                  scalers=['identity'],
                  deg=deg, batch_norm=False)

In [ ]:
from gsn import GSN
from gsn import prepare_GSN_dataset
prepare_GSN_dataset(dataset, max_cycle=8)

In [ ]:
from torchdrug import core, models, tasks

model = GSN(input_dim=dataset.node_feature_dim,
                  hidden_dim=256, num_layer=3,
                  edge_input_dim=dataset.edge_feature_dim,
                  num_relation=dataset.num_bond_type,
                  batch_norm=False)
task = tasks.GCPNGeneration(model, dataset.atom_types, max_edge_unroll=12,
                            max_node=38, criterion="nll")

In [ ]:
from torch import nn, optim
optimizer = optim.Adam(task.parameters(), lr = 1e-3)
solver = core.Engine(task, dataset, None, None, optimizer,
                     batch_size=128, log_interval=10)

solver.train(num_epoch=1)